In [2]:
import os
src = "../input/mthv2-wrl/MTH1200/label_char/GL_1047_1_11.txt"
f_char = open(src)
text = f_char.read()
text = [i.split(' ') for i in text.split('\n')]
print(text[0])
# [character, left_col, up_row, right_col, down_row]
tex  = text[0]

In [3]:
import cv2
import matplotlib.pyplot as plt
img = cv2.imread('../input/mthv2-wrl/MTH1200/img/GL_1047_1_11.jpg')
print("image shape is : {}".format(img.shape))
plt.imshow(img)

tex = [int(float(i)) for idi,i in enumerate(tex) if(idi>0)]
print(tex)
# [row, column]
crop = img[tex[1]:tex[3], tex[0]:tex[2]]
plt.imshow(crop)

**Convert gray-scale image to rgb**

In [21]:
img = img[:,:,0]
print("image shape is : {}".format(len(img.shape)))
backtorgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
print("image shape is convert to: {}".format(len(backtorgb.shape)))
plt.imshow(backtorgb)

In [ ]:
out_root = "../working/"
outputs = os.listdir("../working/")
#os.mkdir("../working/text[0][0]/")
#cv2.imwrite("../working/"+text[0][0]+".png",crop)
subfiles = os.listdir(out_root)
print(subfiles)

In [48]:
# read characters' indexes from a txt
# then extract corresbonding image
def read_1_img(txt_src, img_src, out_root):
    # check if txt_src and img_src exist
    if(not(os.path.isfile(txt_src) and os.path.isfile(img_src))):
        return
    
    # read from source
    f_char = open(txt_src)
    img = cv2.imread(img_src)
    
    # adding channels for gray-scale map
    if len(img.shape) < 3:
        img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
        
    text = f_char.read()
    text = [i.split(' ') for i in text.split('\n')][:-1]
    for tex in text:
        character = tex[0]
        # print(tex, flush = True)
        tex = [int(float(i)) for idi,i in enumerate(tex) if(idi>0)]
        crop = img[tex[1]:tex[3], tex[0]:tex[2]]
        
        # 裁剪区域为空
        if crop.size == 0: 
            continue
            
        # new character
        if character not in os.listdir(out_root):
            new_dir = out_root + character + "/"
            os.mkdir(new_dir)
            #print(new_dir + character + "0.png")
            cv2.imwrite(new_dir + character + "0.jpg", crop)
        else:
            now_dir = out_root + character + "/"
            nums = len(os.listdir(now_dir))
            #print(now_dir + character + str(nums) + ".png")
            cv2.imwrite(now_dir + character + str(nums) + ".jpg", crop)
# read_1_img("../input/mthv2-wrl/MTH1000/label_char/005.txt", '../input/mthv2-wrl/MTH1000/img/005.jpg', "../working/")

img_roots = ["../input/mthv2-wrl/MTH1000/img/",
             "../input/mthv2-wrl/MTH1200/img/",
             "../input/mthv2-wrl/TKH/img"]
label_roots = ["../input/mthv2-wrl/MTH1000/label_char/", 
               "../input/mthv2-wrl/MTH1200/label_char/",
               "../input/mthv2-wrl/TKH/label_char"]
out_root = "../working/"

# generate images from every sub_file
for i in range(len(img_roots)):
    
    img_root = img_roots[i]
    print(img_root)
    label_root = label_roots[i]
    print(label_root)
    
    img_list = os.listdir(img_root)
    label_list = os.listdir(label_root)

    for i,item in enumerate(img_list):
        # print process
        if(int(len(img_list)*0.25) == i):
            print("25%")
        elif(int(len(img_list)*0.5) == i):
            print("50%")
        elif(int(len(img_list)*0.75) == i):
            print("75%")
        name = item[:-4]
        img_src = img_root + item
        label_src = label_root + name + ".txt"
    
        #print(item, flush = True)
        read_1_img(label_src, img_src, out_root)
    print("100%")

In [71]:
!ls

In [70]:
print(len(os.listdir("/kaggle/working/")))

In [ ]:
# markdown commond for downloading a file
# <a href="./output.zip"> Download File </a>

<a href="./output.zip"> Download File </a>

Divide raw dataset as training and testing

In [50]:
# divide as train as test
import random
import shutil

# filter out non-chinese characters
categories = [i for i in os.listdir("./") if '\u4e00' <= i <= '\u9fa5']
print("pure chinese:", len(categories))

# configs
train_root = "./train"
test_root = "./test"
root = "./"
proportion = 5 # train : test = 4 : 1

# make test root
os.mkdir(test_root)
os.mkdir(train_root)

# create sub_files for every class
for i in categories:
    os.mkdir(test_root + "/" + i)
    os.mkdir(train_root + "/" + i)

# train:text = 4:1
# move away part of images to generate a train and test dataset
for i, chara in enumerate(categories):
    # print process
    if(int(len(categories)*0.25) == i):
        print("25%")
    elif(int(len(categories)*0.5) == i):
        print("50%")
    elif(int(len(categories)*0.75) == i):
        print("75%")
    
    # total amount of classes
    files = os.listdir(root + chara +"/")
    total_len = len(files)
    
    # move the front 1/5 images to anthor file
    test_len = int(total_len/proportion)
    if(test_len == 0):# if the file only has 1 img, allocate it to training set
        continue
    #print([files[i] for i in range(train_len)])
    for i in range(test_len):
        src = root + chara + "/" + chara + str(i) + ".jpg"
        dst = test_root + "/" + chara + "/"
        shutil.move(src, dst)
        # print(src)
        # print(dst)
    # move to train
    left = os.listdir(root + chara +"/")
    for i in left:
        src = root + chara + "/" + i
        dst = train_root + "/" + chara + "/"
        shutil.move(src, dst)
        # print(src)
        # print(dst)
    
print("end move")

In [68]:
!zip -q -r train.zip ./train

<a href="./train.zip"> Download File </a>

In [30]:
# delete non-chinese characters' files
for file in os.listdir("./"):
    if(file == '__notebook_source__.ipynb'):
        continue
    if(file not in categories):
        shutil.rmtree("./" + file) 
print(len(os.listdir("./")))

In [69]:
!zip -q -r test.zip ./test

<a href="./test.zip"> Download test.zip File </a>